# Areas of Interest (AOIs)
This notebook creates GeoJSON files from KML and KMZ files covering Areas of Interest, builds the `aoi` table within the SpataiLite database, and populates the table.

### Import libraries

In [1]:
# Basic stack
import os
import shutil
import warnings
from glob import glob

# Web stack
import json

# Database stack
import sqlite3

# Data Science stack
import pyproj
from pyproj import CRS
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import utm
import pandas as pd
import geopandas as gpd
import shapely
from shapely import to_geojson
from shapely.ops import transform
from shapely.geometry import Polygon
from fiona.drvsupport import supported_drivers
import folium

### Environmental settings

In [2]:
warnings.filterwarnings('ignore')

### User defined variables

In [3]:
dir_kml = "../../data/kml/aois"
dir_kmz = "../../data/kmz/aois"
dir_geojson = "../../data/geojson/aois"
db = "../../db.sqlite3"

### User defined functions

In [15]:
def select_data(c, columns):
    """ Selects all data from the `aois` table for review

        C - A cursor
        COLUMNS - Columns of interest
    """
    columns = ', '.join(columns)
    sql_string = f"SELECT {columns}, AsText(geom) FROM aois"
    c.execute(sql_string)
    return c.fetchall()

def update_aoi(c, idd, name, utm, sqkm, geom):
    """ Insert/update the `aois` SQLite Data Table with a new Area of Interest.

        C - A cursor
        AOI ID - The AOI ID
        NAME - The location name
        UTM - The UTM Zone as an EPSG code
        SQKM - Square kilometers
        GEOM - The area of interest's geometry
    """
    sql_string = (f"INSERT INTO aoi(id, name, utm, sqkm, geom) \n"
                     f"VALUES ({idd}, \"{name}\", \"{utm}\", {sqkm}, ST_GeomFromText(\"{geom}\", 4326))")
    c.execute(sql_string)

def utm_and_area(aoi):
    """ Determines the UTM Zone, as an EPSG code, of an Area of Interest
            as well as culates the area, in square kilometers, of this
            Area of Interest.

        AOI - Area of Interest as a Shapely Polygon
    """
    minx, miny, maxx, maxy = aoi.bounds
    
    utm_crs_list = query_utm_crs_info(
        datum_name = "WGS 84",
        area_of_interest=AreaOfInterest(
            west_lon_degree = minx,
            south_lat_degree = miny,
            east_lon_degree = maxx,
            north_lat_degree = maxy,
        ),
    )

    if len(utm_crs_list) == 1:
        print("One zone, {}, was identified and will be used for calculations.".format(utm_crs_list[0].name))
    else:
        print("Multiple zones were identified. {} will be used for calculations.".format(utm_crs_list[0].name))
        alt_utms = [utm_crs.name for utm_crs in utm_crs_list][1:]
        print("\tOther options were: {}".format(alt_utms))
    
    utm_crs = CRS.from_epsg(utm_crs_list[0].code)
    proj_wgs2utm = pyproj.Transformer.from_crs(pyproj.CRS('EPSG:4326'), utm_crs, always_xy=True).transform
    sqkm = transform(proj_wgs2utm, aoi).area / 1_000_000
    return utm_crs, sqkm

def geojson_to_row(geojson):
    """ Creates a GeoDataFrame from a GeoJSON file then adds columns for
            the AOI ID, location name, UTM Zone (as an EPSG code), and
            area in square kilometers.

        Is dependent on the UTM_AND_AREA function above.

        GEOJSON - A GeoJSON file
    """
    try:
        gdf = gpd.read_file(geojson, crs='EPSG:4326')
        root_name = geojson.split('/')[-1]
        gdf['id'] = root_name.split('.')[0].split('-')[0]
        gdf['name'] = root_name.split('.')[0].split('-')[1]
        utm_crs, sqkm = utm_and_area(gdf['geometry'][0])
        gdf['utm'] = utm_crs
        gdf['sqkm'] = round(sqkm, 2)
        return gdf
    except Exception as e:
        print("Failed on {} due to: {}".format(geojson, e))
        pass

def kml_to_geojson(kml, out_dir):
    """ Converts a KML file to GeoJSON file.

        KML - KML file
        OUT DIR - Output directory
    """
    supported_drivers['KML'] = 'rw'

    kml_name = kml.split('\\')[-1]
    gdf = gpd.read_file(kml, driver='KML')
    if len(gdf["geometry"]) == 1:
        kml_shape = to_geojson(gdf['geometry'][0])

        root_name = kml_name.split('.')[0]
        geojson_file = "{}/{}.geojson".format(out_dir, root_name)
        with open(geojson_file, "w") as f:
            f.write(kml_shape)
            f.close()
        
        with open(geojson_file, "r") as f:
            aoi = json.loads(f.read())
            f.close()

        return geojson_file
            
    elif len(gdf["geometry"]) > 1:
        print("More than one geometry found in your {} KML. Passing...".format(kml_name))
        pass

### Convert AOI files to GeoJSON
Note, that while many files can be handled, strange file naming conventions will likely result in errors. Ideally, files should not be longer than 13 characters, not use spaces or any other special characters, and avoid `-`. Also note that only KML and KMZ files are supported, not XLSX.

In [5]:
aoi_kmls = glob(dir_kml + "/*.kml")
aoi_kmzs = glob(dir_kmz + "/*.kmz")
aois = aoi_kmls + aoi_kmzs

if not os.path.exists(dir_geojson):
    os.makedirs(dir_geojson)

aoi_geojsons = [kml_to_geojson(aoi, dir_geojson) for aoi in aois]

More than one geometry found in your 0135-RemoteSensingSI-NWFSC-WestofSanJuans.kml KML. Passing...


### Read GeoJSONs into GeoDataFrame
Determine which EPSG, or WKID, cover the Area of Interest the best while reading the GeoJSON file in as a GeoDataFrame. Show if there are any issues with locally projecting to the end user. Reproject all GeoDataFrames from their local, UTM projection to EPSG 4326, better known as WGS84. Concactenate the GeoDataFrames into a single GeoDataFrame. Show the GeoDataFrame's head to the end user.

In [7]:
aoi_dfs = []
for aoi_geojson in aoi_geojsons:
    try:
        aoi_dfs.append(geojson_to_row(aoi_geojson))
    except Exception as e:
        print(f"Failed on {aoi_geojson} with exception {e}")

aois_4326 = []
for aoi_df in aoi_dfs:
    try:
        aois_4326.append(aoi_df.to_crs("EPSG:4326"))
    except Exception as e:
        print(f"Failed on {aoi_df} with exception {e}")

gdf = pd.concat(aois_4326, ignore_index=True)
corrected_columns = ['id', 'name', 'utm', 'sqkm', 'geometry']
gdf = gdf[corrected_columns]
gdf.head()

One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 5N, was identified and will be used for calculations.
Multiple zones were identified. WGS 84 / UTM zone 5N will be used for calculations.
	Other options were: ['WGS 84 / UTM zone 6N']
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identified and will be used for calculations.
One zone, WGS 84 / UTM zone 19N, was identif

,id,name,utm,sqkm,geometry
0,0001,Kenai,EPSG:32605,416.53,"POLYGON Z ((-151.59303 60.32094 0, -151.43153 ..."
1,0002,Kalgin Island,EPSG:32605,884.65,"POLYGON Z ((-151.73795 60.71243 0, -151.69676 ..."
2,0003,Trading Bay,EPSG:32605,529.37,"POLYGON Z ((-151.34361 61.01015 0, -151.22669 ..."
3,0004,Tuxedni,EPSG:32605,438.72,"POLYGON Z ((-152.60081 60.02992 0, -152.59393 ..."
4,0005,Upper CI,EPSG:32605,1871.28,"POLYGON ((-150.67303 60.95571, -151.00321 61.1..."


### Show the shape of that GeoDataFrame
This is helpful for troublehsooting later.

In [8]:
gdf.shape

(79, 5)

### Filter to only Polygons, force to just Polygon
The `aoi` table will have a geometry of Polygon; so we filter to only Polygons, excluding Multi Polygons. Some returned Polygons are veritcally, or Z, enabled. FOrce these to be just Polygons.

In [9]:
print(f"Your initial GeoDataFrame had the shape {gdf.shape}")
gdf = gdf[gdf['geometry'].type == 'Polygon']
gdf['geometry'] = gdf.force_2d()
print(f"Your resulting GeoDataFrame has the shape {gdf.shape}")

gdf.head()

Your initial GeoDataFrame had the shape (79, 5)
Your resulting GeoDataFrame has the shape (71, 5)


,id,name,utm,sqkm,geometry
0,0001,Kenai,EPSG:32605,416.53,"POLYGON ((-151.59303 60.32094, -151.43153 60.6..."
1,0002,Kalgin Island,EPSG:32605,884.65,"POLYGON ((-151.73795 60.71243, -151.69676 60.5..."
2,0003,Trading Bay,EPSG:32605,529.37,"POLYGON ((-151.34361 61.01015, -151.22669 60.9..."
3,0004,Tuxedni,EPSG:32605,438.72,"POLYGON ((-152.60081 60.02992, -152.59393 60.0..."
4,0005,Upper CI,EPSG:32605,1871.28,"POLYGON ((-150.67303 60.95571, -151.00321 61.1..."


### Drop table
This is for demonstration and troubleshooting purposes.

In [10]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS aois''')
conn.commit()
conn.close()

### Connect to databse and create `aoi` table

In [11]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS aoi(
        id INTEGER PRIMARY KEY,
        name VARCHAR(50),
        requestor VARCHAR(25),
        utm VARCHAR(10),
        sqkm NUMERIC(10, 2)
    )
''')

c.execute('''SELECT AddGeometryColumn('aoi', 'geom', 4326, 'POLYGON')''')

conn.commit()
conn.close()

### Using the GeoDataFrame, insert the AOIs into the `aoi` table

In [16]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in gdf.iterrows():
    try:
        update_aoi(c, row['id'], row['name'], row['utm'], row['sqkm'], row['geometry'])
    except Exception as e:
        print("Exception: {} was raised for AOI ID {}".format(e, row['id']))
print("Done updating AOI table!")

conn.commit()
conn.close()

Done updating AOI table!


### Select newly inserted AOIs, make a GeoDataFrame for validation

In [17]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

main_columns = list(gdf.columns)[:-1]
main_columns = ', '.join(main_columns)
df = pd.read_sql_query(f"SELECT {main_columns}, AsText(geom) FROM aoi", conn)
df = df.rename(columns={'AsText(geom)': 'geometry'}, errors='raise')
df['geometry'] = shapely.wkt.loads(df['geometry'])
gdf = gpd.GeoDataFrame(df, geometry='geometry')

conn.commit()
conn.close()

gdf.head()

,id,name,utm,sqkm,geometry
0,1,Kenai,EPSG:32605,416.53,"POLYGON ((-151.59302 60.32094, -151.43153 60.6..."
1,2,Kalgin Island,EPSG:32605,884.65,"POLYGON ((-151.73795 60.71243, -151.69676 60.5..."
2,3,Trading Bay,EPSG:32605,529.37,"POLYGON ((-151.34361 61.01015, -151.22669 60.9..."
3,4,Tuxedni,EPSG:32605,438.72,"POLYGON ((-152.60081 60.02992, -152.59393 60.0..."
4,5,Upper CI,EPSG:32605,1871.28,"POLYGON ((-150.67303 60.95571, -151.00321 61.1..."


### Plot Areas of Interest on an Interactive Map

In [19]:
def style_function(hex_value):
    return {'color': hex_value, 'fillOpacity': 0}

# Add OpenStreetMap as a basemap
map = folium.Map()
folium.TileLayer('openstreetmap').add_to(map)

# Create a GeoJson layer from the response_geojson and add it to the map
folium.GeoJson(
    gdf['geometry'].to_json(),
    style_function = lambda x: style_function('#0000FF')
).add_to(map)

# Zoom to collected images
map.fit_bounds(map.get_bounds(), padding=(100, 100))

# Display the map
map

# End